`python` 기능을 확장해 주는 `module`을 불러 들임 (일부 기능만 사용될 수도 있음)

In [ ]:
import numpy as np  # 배열, 행렬 관련 기능
import numpy.linalg as na  # 선형대수 (벡터, 행렬) 관련 기능
import matplotlib.pyplot as plt  # 그래프 관련 기능
import scipy.integrate as si  # 적분 관련 기능
import sympy as sy  # 기호 연산 기능
import sympy.plotting as splot
import IPython.display as disp  # 웹페이지 표시 기능
sy.init_printing()  # 기호 연산 결과 표시 기능 준비

## 예제 06.001

### 문제에서 주어진 변수

#### 보의 길이

In [ ]:
L_AB_m = sy.symbols('L_AB_m', positive=True)

s_d = {L_AB_m: 4}

#### 하중

In [ ]:
w_AB_N_m = sy.Symbol('w_AB_N_m', real=True)
s_d[w_AB_N_m] = -2000

#### 재료

In [ ]:
E_Pa = sy.Symbol('E_Pa', positive=True)

s_d[E_Pa] = 200e9

#### 단면

W 200 $\times$ 100 에 관련된 자료는 p.608 참고

In [ ]:
A_m2, I_m4, h_m = sy.symbols('A_m2, I_m4, h_m', positive=True)
A_mm2 = A_m2 * 1e-6
I_mm4 = I_m4 * 1e-12
h_mm = h_m * 1e3

값 계산을 위해 준비

In [ ]:
s_d.update({A_mm2: 12700.0,
            I_mm4: 113e6,
            h_mm: 229,})

s_d.update({A_m2: s_d[A_mm2] * 1e-6,
            I_m4: s_d[I_mm4] * 1e-12,
            h_m: s_d[h_mm] * 1e-3,})

준비된 값을 확인

In [ ]:
s_d

### A, B, C, D 지점의 x 좌표

A 점에서 $x=0$ 으로 한다.

In [ ]:
x_m = sy.Symbol('x_m', nonnegative=True)
x_A_m = 0
x_B_m = float(x_A_m + L_AB_m.subs(s_d))

x 지점에서의 굽힘모멘트

In [ ]:
M_Nm = -w_AB_N_m * x_m * (x_m/2)
M_Nm

p. 236 eq. 6.3b

$$
EI\nu''=M
$$

$\nu$ 와 그 미분을 준비

In [ ]:
EInu_Nm3 = sy.Function('EI\\nu_Nm3')
# http://docs.sympy.org/latest/modules/solvers/ode.html#dsolve
# http://docs.sympy.org/latest/tutorial/calculus.html
EInu_pp_Nm = sy.Derivative(EInu_Nm3(x_m), x_m, x_m)
EInu_Nm3, EInu_pp_Nm

미분방정식 등호의 왼쪽

In [ ]:
left_side = EInu_pp_Nm
left_side

등호의 오른쪽

In [ ]:
right_side = M_Nm
right_side

eq. 6.3b

In [ ]:
defl_deq = sy.Eq(left_side, right_side )
defl_deq

미분방정식을 두번 적분한 결과

In [ ]:
EInu_Nm3_sol_eq = sy.dsolve(defl_deq, EInu_Nm3(x_m))
EInu_Nm3_sol_eq

위 결과에서 등호의 오른쪽은 다음과 같다.

In [ ]:
EInu_Nm3_sol = EInu_Nm3_sol_eq.rhs
EInu_Nm3_sol

여기서 적분 상수를 결정해야 한다.

경계조건 두가지 (처짐, 기울기) 가운데 기울기 조건을 적용한다.

$$ \left. \frac{d\nu}{dx} \right\rvert _{x=L}=0 $$

In [ ]:
# SymPy Dev Team, Derivatives, Calculus, SymPy Tutorial, 2017 Jul 27, http://docs.sympy.org/latest/tutorial/calculus.html#derivatives
slope_eq = sy.Eq(sy.diff(EInu_Nm3_sol, x_m).subs(x_m, L_AB_m), 0)
slope_eq

In [ ]:
defl_eq = sy.Eq(EInu_Nm3_sol.subs(x_m, L_AB_m), 0)
defl_eq

위 두 방정식을 연립하여 풂

In [ ]:
const_sol = sy.solve((slope_eq, defl_eq))
const_sol

적분상수를 $EI\nu$ 식에 대입함

In [ ]:
EInu_Nm3_sol.subs(const_sol[0])